In [0]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.datasets import CIFAR10

In [0]:
def conv3x3(in_channel, out_channel, stride=1):
    return nn.Conv2d(in_channel, out_channel, 3, stride=stride, padding=1, bias =False)

In [0]:
class residual_block(nn.Module):
    def __init__(self, in_channel, out_channel, same_shape=True):
        super(residual_block, self).__init__()
        self.same_shape = same_shape
        stride = 1 if self.same_shape else 2
        
        self.conv1 = conv3x3(in_channel, out_channel, stride=stride)
        self.bn1 = nn.BatchNorm2d(out_channel)
        
        self.conv2 = conv3x3(out_channel, out_channel)
        self.bn2 = nn.BatchNorm2d(out_channel)
        
        if not self.same_shape:
            self.conv3 = nn.Conv2d(in_channel, out_channel ,1, stride=stride)
            
    def forward(self,x):
        out = self.conv1(x)
        out = F.relu(self.bn1(out), True)
        out = self.conv2(out)
        out = F.relu(self.bn2(out), True)
        
        if not self.same_shape:
            x = self.conv3(x)
            
        return F.relu(x + out, True)

In [4]:
test_net = residual_block(32,32)
test_x = Variable(torch.zeros(1,32,96,96))
print('input:{}'.format(test_x.shape))
test_y = test_net(test_x)
print('output:{}'.format(test_y.shape))

input:torch.Size([1, 32, 96, 96])
output:torch.Size([1, 32, 96, 96])


In [5]:
test_net = residual_block(3, 32, False)
test_x = Variable(torch.zeros(1, 3, 96, 96))
print('input:{}'.format(test_x.shape))
test_y = test_net(test_x)
print('output:{}'.format(test_y.shape))

input:torch.Size([1, 3, 96, 96])
output:torch.Size([1, 32, 48, 48])


In [0]:
class resnet(nn.Module):
    def __init__(self, in_channel, num_classes, verbose=False):
        super(resnet, self).__init__()
        self.verbose = verbose
        
        self.block1 = nn.Conv2d(in_channel, 64, 7, 2)
        
        self.block2 = nn.Sequential(
            nn.MaxPool2d(3,2),
            residual_block(64,64),
            residual_block(64,64)
        )
        
        self.block3 = nn.Sequential(
            residual_block(64, 128, False),
            residual_block(128, 128)
        )
        
        self.block4 = nn.Sequential(
            residual_block(128, 256, False),
            residual_block(256, 256)
        )
        
        self.block5 = nn.Sequential(
            residual_block(256, 512, False),
            residual_block(512,512),
            nn.AvgPool2d(3)
        )
        
        self.classifier = nn.Linear(512, num_classes)
    
    def forward(self, x):
        x = self.block1(x)
        if self.verbose:
            print('block 1 output :{}'.format(x.shape))
        
        x = self.block2(x)
        if self.verbose:
            print('block 2 output :{}'.format(x.shape))
            
        x = self.block3(x)
        if self.verbose:
            print('block 3 output :{}'.format(x.shape))
            
        x = self.block4(x)
        if self.verbose:
            print('block 4 output :{}'.format(x.shape))
            
        x = self.block5(x)
        if self.verbose:
            print('block 5 output :{}'.format(x.shape))
        
        x = x.view(x.shape[0], -1)
        x = self.classifier(x)
        return x

In [7]:
test_net = resnet(3, 10, True)
test_x = Variable(torch.zeros(1,3,96,96))
print('input : {}'.format(test_x.shape))
test_y = test_net(test_x)
print('output : {}'.format(test_y.shape))

input : torch.Size([1, 3, 96, 96])
block 1 output :torch.Size([1, 64, 45, 45])
block 2 output :torch.Size([1, 64, 22, 22])
block 3 output :torch.Size([1, 128, 11, 11])
block 4 output :torch.Size([1, 256, 6, 6])
block 5 output :torch.Size([1, 512, 1, 1])
output : torch.Size([1, 10])


In [8]:
from Myutils import train

def data_tf(x):
    x = np.array(x, dtype='float32') / 255
    x = (x - 0.5) / 0.5
    x = x.transpose((2,0,1))
    x = torch.from_numpy(x)
    return x

train_set = CIFAR10('./data', train=True, transform=data_tf ,download=True)
train_data = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)

test_set = CIFAR10('./data', train=False, transform=data_tf)
test_data = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False)

net = resnet(3,10)
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

Files already downloaded and verified


In [10]:
train(net, train_data, test_data, 20, optimizer, criterion)

/content/Myutils.py:70: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  im = Variable(im.cuda(), volatile=True)
/content/Myutils.py:71: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  label = Variable(label.cuda(), volatile=True)


Epoch 0. Train Loss: 1.446816, Train Acc: 0.477621, Valid Loss: 1.829497, Valid Acc: 0.387263, Time 00:00:42
Epoch 1. Train Loss: 1.109312, Train Acc: 0.607497, Valid Loss: 1.410564, Valid Acc: 0.522449, Time 00:00:44
Epoch 2. Train Loss: 0.955399, Train Acc: 0.663743, Valid Loss: 1.000580, Valid Acc: 0.647152, Time 00:00:44
Epoch 3. Train Loss: 0.841582, Train Acc: 0.704424, Valid Loss: 1.223143, Valid Acc: 0.594739, Time 00:00:44
Epoch 4. Train Loss: 0.748967, Train Acc: 0.736193, Valid Loss: 1.689838, Valid Acc: 0.503857, Time 00:00:44
Epoch 5. Train Loss: 0.668684, Train Acc: 0.764047, Valid Loss: 0.902645, Valid Acc: 0.696203, Time 00:00:44
Epoch 6. Train Loss: 0.588507, Train Acc: 0.793159, Valid Loss: 1.142933, Valid Acc: 0.637856, Time 00:00:44
Epoch 7. Train Loss: 0.532220, Train Acc: 0.812360, Valid Loss: 1.023406, Valid Acc: 0.667919, Time 00:00:44
Epoch 8. Train Loss: 0.468078, Train Acc: 0.837036, Valid Loss: 1.146881, Valid Acc: 0.642405, Time 00:00:44
Epoch 9. Train Loss

In [0]:
class bottle_block(nn.Module):
    def __init__(self, in_channel, middle_channel, same_shape=True):
        super(bottle_block, self).__init__()
        self.same_shape = same_shape
        
        self.conv1 = nn.Conv2d(in_channel, middle_channel, 1)
        self.bn1 = nn.BatchNorm2d(middle_channel)
        
        self.conv2 = nn.Conv2d(middle_channel, middle_channel, 3)
        self.bn2 = nn.BatchNorm2d(middle_channel)
        
        self.conv3 = nn.Conv2d(middle_channel, in_channel, 1)
        
    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(self.bn1(out), True)
        out = self.conv2(out)
        out = F.relu(self.bn2(out), True)
        out = self.conv3(out)
        
        return F.relu(x + out, True)